In [1]:
!pip install -q datasets==3.2.0 bitsandbytes==0.45.1 evaluate==0.4.3 seqeval==1.2.2 peft==0.14.0 torch==2.5.1+cu124 transformers==4.46.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/66

In [2]:
import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    RobertaForTokenClassification,
    RobertaTokenizerFast,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training,
    PeftModel
)

In [27]:
dataset = load_dataset("conll2003" , split = 'train[:5000]')

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [28]:
dataset

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 5000
})

In [29]:
dataset['ner_tags'][:3]

[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0]]

In [30]:
ner_list = dataset.features['ner_tags'].feature.names

label2id = { tag : i for i, tag in enumerate(ner_list) }
id2label = { i : tag for i, tag in enumerate(ner_list) }

In [3]:
# For NER with RoBERTa, use add_prefix_space=True to prepend spaces to words
# for proper tokenization, unlike WordPiece-based models like DistilBERT.

model_name = 'FacebookAI/roberta-base'

tokenizer = RobertaTokenizerFast.from_pretrained(model_name , add_prefix_space = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [31]:
def tokenize_and_align_labels(examples, label_all_tokens = True):

    """
    Tokenize and align NER labels with subword tokens.
    For example, "i love NewYork" -> has tags [0, 0, 5].
    After tokenization, ['i', 'love', 'New', '##York'] -> has tags [0, 0, 5, 5].
    Subwords like 'New' and '##York' inherit the label of the original word.
    Special tokens (e.g., [CLS], [SEP]) are labeled as -100 to ignore them in the loss function.
    The `label_all_tokens` flag determines if all subwords or only the first subword of a word get the label.
    """

    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation = True,
        is_split_into_words = True # tells the tokenizer that the dataset is tokens not sentences
    )

    labels = []

    for i, label in enumerate( examples[ "ner_tags" ] ):

        word_ids = tokenized_inputs.word_ids( batch_index = i ) # word_ids returns somthing like [0,1,2,2,...], if it comes to a sub token it returns the same id
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:

            # Special tokens have a word id that is None. We set
            # the label to -100 so they are automatically ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)

            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append( label[ word_idx ] )

            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

In [32]:
tokenized_dataset = dataset.map(
    tokenize_and_align_labels ,
    batched = True ,
    remove_columns = dataset.column_names ,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [33]:
train_val_dataset = tokenized_dataset.shuffle( 1000 ).train_test_split( .15 )
train_val_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 750
    })
})

In [34]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit = True ,
    llm_int8_threshold = 6.0 ,
)

In [35]:
base_model = RobertaForTokenClassification.from_pretrained(
    model_name ,
    quantization_config = quantization_config ,
    num_labels = len(ner_list) ,
    label2id = label2id ,
    id2label = id2label ,
)

model = prepare_model_for_kbit_training( base_model )

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Extract the names of linear layers (e.g., 'q_lin', 'k_lin', etc.)
for name, module in base_model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(name)

roberta.encoder.layer.0.attention.self.query.base_layer
roberta.encoder.layer.0.attention.self.query.lora_A.default
roberta.encoder.layer.0.attention.self.query.lora_B.default
roberta.encoder.layer.0.attention.self.key.base_layer
roberta.encoder.layer.0.attention.self.key.lora_A.default
roberta.encoder.layer.0.attention.self.key.lora_B.default
roberta.encoder.layer.0.attention.self.value.base_layer
roberta.encoder.layer.0.attention.self.value.lora_A.default
roberta.encoder.layer.0.attention.self.value.lora_B.default
roberta.encoder.layer.0.attention.output.dense.base_layer
roberta.encoder.layer.0.attention.output.dense.lora_A.default
roberta.encoder.layer.0.attention.output.dense.lora_B.default
roberta.encoder.layer.0.intermediate.dense.base_layer
roberta.encoder.layer.0.intermediate.dense.lora_A.default
roberta.encoder.layer.0.intermediate.dense.lora_B.default
roberta.encoder.layer.0.output.dense.base_layer
roberta.encoder.layer.0.output.dense.lora_A.default
roberta.encoder.layer.0.ou

In [36]:
lora_setup = LoraConfig(
    r = 8,
    lora_alpha = 4,
    lora_dropout = 0.05,
    bias = 'none',
    task_type = TaskType.TOKEN_CLS,
    target_modules = ['dense', 'query', 'key', 'value', "dense"],
)

model = get_peft_model( model , lora_setup )

print( model.print_trainable_parameters() )

trainable params: 1,334,025 || all params: 125,395,986 || trainable%: 1.0638
None


In [41]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="pt")

In [38]:
metric = evaluate.load( 'seqeval' )

def compute_metrics(pred_labels):

    predictions, labels = pred_labels
    predictions = np.argmax(predictions, axis=-1)

    # Remove ignored indeces (special tokens)
    true_predictions = [
        [ner_list[p]
         for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [ner_list[l]
         for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions = true_predictions, references = true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [43]:
train_args = TrainingArguments(
    output_dir = "./results",
    save_strategy = 'epoch',
    num_train_epochs = 1,
    learning_rate = 3e-4,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps= 2,
    weight_decay = 0.01, # l2 regularization
    eval_strategy = 'epoch',
    logging_dir="./logs",
    logging_strategy = 'steps',
    logging_steps = 50,
    bf16 = True,
    load_best_model_at_end = True,
    lr_scheduler_type = 'linear',
    warmup_ratio = 0.05,
    max_grad_norm = 1.0,
    group_by_length = True,
)

In [44]:
trainer = Trainer(
    model = model ,
    args = train_args ,
    train_dataset = train_val_dataset['train'] ,
    eval_dataset = train_val_dataset['test'] ,
    data_collator = data_collator ,
    compute_metrics = compute_metrics ,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to 

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.054100,0.334793,0.546309,0.570294,0.558044,0.903093


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=133, training_loss=1.3319467243395353, metrics={'train_runtime': 68.2998, 'train_samples_per_second': 62.226, 'train_steps_per_second': 1.947, 'total_flos': 44919845752104.0, 'train_loss': 1.3319467243395353, 'epoch': 1.0})

In [51]:
# model.save_pretrained("./roberta_ner_model")

In [47]:
text = {"tokens" : ['mike','goes','to','his','grandma','everyday']}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get the device

tokenized = tokenizer(text,
                      is_split_into_words=True,
                      padding=True,
                      truncation=True,
                      max_length=128,
                      return_tensors = 'pt')

tokenized = tokenized.to(device)

model.eval()

with torch.no_grad():
    logits = model(**tokenized).logits
    proba = torch.softmax(logits, dim = -1)
    idx = torch.argmax(proba, dim = -1)[0]

# Get the word IDs to align predictions with original words
word_ids = tokenized.word_ids()

# Iterate through the word IDs and print the corresponding NER tag and word
for i, word_id in enumerate(word_ids):
    # Ignore special tokens (word_id is None)
    if word_id is not None:
        # Get the predicted NER tag for the word
        predicted_tag_id = idx[i].item()  # Convert tensor to integer

        # Print the NER tag and the corresponding word
        print(f'{ner_list[predicted_tag_id]} : {text[word_id]}')

B-PER : mike
B-PER : mike
O : goes
O : to
O : his
O : grandma
O : everyday


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
